In [72]:
from src.decode import read_source, parse_program, dump_file
from src.funcs import *
from src.analyze import sorted_defs, get_unique_lexems, is_token, inline

In [74]:
AP(CAR, AP(AP(AP(CONS, 1), 2)))

TypeError: <lambda>() missing 1 required positional argument: 'x'